# Оптимизация гиперпараметров

In [13]:
import pandas as pd
import numpy as np
import pygad
import pyswarms as ps
import random
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import joblib
import os
'''
import neat
from neat.config import Config
from neat.genome import DefaultGenome
from neat.reproduction import DefaultReproduction
from neat.species import DefaultSpeciesSet
from neat.stagnation import DefaultStagnation
'''

'\nimport neat\nfrom neat.config import Config\nfrom neat.genome import DefaultGenome\nfrom neat.reproduction import DefaultReproduction\nfrom neat.species import DefaultSpeciesSet\nfrom neat.stagnation import DefaultStagnation\n'

In [14]:
def generate_dataset(num_samples):
    data = []
    for _ in range(num_samples):

        object1_state = [
            random.choice([0, 1]),  # Бинарный признак (например, активен/не активен)
            random.choice(['Круг', 'Квадрат', 'Треугольник']),  # Номинальный признак (форма объекта)
            random.choice(['Север', 'Юг', 'Восток', 'Запад']),  # Порядковый признак (направление)
            random.uniform(0, 360)  # Количественный признак (угол поворота)
        ]

        object2_state = [
            random.choice([0, 1]),  # Бинарный признак
            random.choice(['Круг', 'Квадрат', 'Треугольник']),  # Номинальный признак
            random.choice(['Север', 'Юг', 'Восток', 'Запад']),  # Порядковый признак
            random.uniform(0, 360)  # Количественный признак
        ]

        # Генерация коллизии (Да/Нет)
        collision = random.choice(['Да', 'Нет'])

        # Добавление строки в данные
        data.append(object1_state + object2_state + [collision])

    # Создание DataFrame
    columns = [
        'Объект 1 Бинарный', 'Объект 1 Форма', 'Объект 1 Направление', 'Объект 1 Угол',
        'Объект 2 Бинарный', 'Объект 2 Форма', 'Объект 2 Направление', 'Объект 2 Угол',
        'Коллизия'
    ]
    return pd.DataFrame(data, columns=columns)

# Параметры для генерации датасетов
sample_sizes = [(30, 100), (100, 500), (500, 1000), (1000, 2000)]

# Генерация 12 датасетов
datasets = []
for sample_range in sample_sizes:
    for _ in range(3):
        num_samples = random.randint(*sample_range)
        dataset = generate_dataset(num_samples)
        datasets.append(dataset)

In [15]:
Model1 = RandomForestClassifier()
Model2 = LogisticRegression(max_iter=1000)

df = datasets[0]

# 1. Загрузка данных
X = df.drop('Коллизия', axis=1)
y = df['Коллизия'].map({'Да': 1, 'Нет': 0})

# 2. Простой препроцессинг
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), ['Объект 1 Угол', 'Объект 2 Угол']),
    ('cat', OneHotEncoder(), ['Объект 1 Форма', 'Объект 2 Форма']),
    ('binary', 'passthrough', ['Объект 1 Бинарный', 'Объект 2 Бинарный'])
])

# 3. Функция для оценки модели
def evaluate(model, X, y):
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    return cross_val_score(pipeline, X, y, cv=3, scoring='accuracy').mean()

# 4. Оптимизация Random Forest
def fitness_rf(ga_instance, solution, solution_idx):
    model = RandomForestClassifier(
        n_estimators=int(solution[0]),
        max_depth=int(solution[1]) if solution[1] > 0 else None,
        random_state=42
    )
    return evaluate(model, X, y)

ga_rf = pygad.GA(
    num_generations=5,
    num_parents_mating=2,
    fitness_func=fitness_rf,
    sol_per_pop=3,
    num_genes=2,
    gene_space=[{'low': 10, 'high': 100}, {'low': 1, 'high': 10}],
    random_seed=42
)
ga_rf.run()

# 5. Оптимизация SVM
def fitness_svm(ga_instance, solution, solution_idx):
    model = SVC(
        C=solution[0],
        gamma=solution[1],
        random_state=42
    )
    return evaluate(model, X, y)

ga_svm = pygad.GA(
    num_generations=5,
    num_parents_mating=2,
    fitness_func=fitness_svm,
    sol_per_pop=3,
    num_genes=2,
    gene_space=[{'low': 0.1, 'high': 10}, {'low': 0.001, 'high': 0.1}],
    random_seed=42
)
ga_svm.run()

# 6. Сохранение и оценка лучших моделей
def save_and_evaluate_best_model(ga_instance, model_type, X, y):
    solution, solution_fitness, _ = ga_instance.best_solution()

    if model_type == 'rf':
        model = RandomForestClassifier(
            n_estimators=int(solution[0]),
            max_depth=int(solution[1]) if solution[1] > 0 else None,
            random_state=42
        )
    else:
        model = SVC(
            C=solution[0],
            gamma=solution[1],
            random_state=42
        )

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    # Обучение на всех данных
    pipeline.fit(X, y)

    # Сохранение модели
    filename = f'best_{model_type}_model.pkl'
    joblib.dump(pipeline, filename)

    # Оценка точности
    accuracy = evaluate(model, X, y)

    return pipeline, accuracy

# Для Random Forest
best_rf, rf_accuracy = save_and_evaluate_best_model(ga_rf, 'rf', X, y)
print(f"\nЛучшая модель Random Forest:")
print(f"n_estimators: {int(ga_rf.best_solution()[0][0])}")
print(f"max_depth: {int(ga_rf.best_solution()[0][1])}")
print(f"Точность: {rf_accuracy:.4f}")
print(f"Модель сохранена в файл: best_rf_model.pkl")

# Для SVM
best_svm, svm_accuracy = save_and_evaluate_best_model(ga_svm, 'svm', X, y)
print(f"\nЛучшая модель SVM:")
print(f"C: {ga_svm.best_solution()[0][0]:.4f}")
print(f"gamma: {ga_svm.best_solution()[0][1]:.4f}")
print(f"Точность: {svm_accuracy:.4f}")
print(f"Модель сохранена в файл: best_svm_model.pkl")

# Сравнение моделей
if rf_accuracy > svm_accuracy:
    print("\nЛучшая модель: Random Forest")
else:
    print("\nЛучшая модель: SVM")

c:\python\.venv\Lib\site-packages\pygad\pygad.py:724: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(
c:\python\.venv\Lib\site-packages\pygad\pygad.py:724: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(



Лучшая модель Random Forest:
n_estimators: 76
max_depth: 6
Точность: 0.5758
Модель сохранена в файл: best_rf_model.pkl

Лучшая модель SVM:
C: 1.6941
gamma: 0.0164
Точность: 0.5758
Модель сохранена в файл: best_svm_model.pkl

Лучшая модель: Random Forest
